In [2]:
from transformers import RobertaTokenizer, RobertaModel
import torch
import pandas as pd
import numpy

/home/codespace/.python/current/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [34]:
def get_embedding_for_a_code_snippet(code_snippet) -> numpy.array:
    tokenizer = RobertaTokenizer.from_pretrained("microsoft/codebert-base")
    model = RobertaModel.from_pretrained("microsoft/codebert-base")

    # Tokenizează codul Python
    inputs = tokenizer(code_snippet, return_tensors="pt")

    # Obține embedding-uri folosind modelul CodeBERT
    with torch.no_grad():
        outputs = model(**inputs)

    # Extrage embedding-urile
    embedding = outputs.last_hidden_state.mean(dim=1).numpy()
    return embedding

In [35]:
# Exemplu de cod Python
code_snippet = """
def add(a, b):
    return a + b
"""

print(get_embedding_for_a_code_snippet(code_snippet))

[[-4.21587586e-01  5.36854155e-02  9.84746441e-02 -8.52071494e-03
  -1.69713825e-01 -7.04039395e-01  5.58284633e-02  3.74027133e-01
   4.20846671e-01  3.17283541e-01 -1.05989158e-01  6.57337427e-01
  -3.31454843e-01 -1.56138495e-01  8.09300601e-01  5.24319224e-02
   2.31235430e-01  2.34686777e-01  1.75770223e-02  3.27220261e-01
  -9.47753787e-02 -5.38964868e-01  7.52315879e-01 -2.91860908e-01
   5.93987286e-01  2.03578666e-01  5.09097464e-02  8.36718380e-01
  -2.63034582e-01  8.14038754e-01 -1.94777250e-01 -3.90525609e-02
   1.49823809e+00 -1.65551469e-01  3.89765590e-01 -1.86520472e-01
  -2.26509944e-01  1.93701431e-01  9.76766124e-02 -3.78127933e-01
  -1.76276729e-01  3.44369084e-01 -9.64827478e-01 -2.09607840e-01
   4.96156842e-01  2.64273882e-01  3.65866721e-01  3.10826749e-02
   3.52302071e-04  4.47413594e-01  3.80040616e-01  2.87721336e-01
  -7.49138713e-01 -2.20499009e-01  1.51437625e-01  3.16667199e-01
  -1.11746275e+00 -5.35684407e-01 -2.04062134e-01 -3.93559694e-01
  -3.93453

In [36]:
def citire_code_snippet_din_fisier(file_path) -> str:
    with open(file_path, 'r') as file:
        code_snippet = file.read()
    return code_snippet

In [37]:
def citire_coduri() -> pd.DataFrame:
    df = pd.read_csv('scop.csv')
    return df

In [38]:
from sklearn.cluster import KMeans
import numpy as np

In [42]:
def get_clasificator():
    dataframe = citire_coduri()
    set_scopuri = set([scop for scop in dataframe['scop']])
    scopuri = [scop for scop in set_scopuri]

    embeddinguri = [get_embedding_for_a_code_snippet(citire_code_snippet_din_fisier(code_snippet_path)) for code_snippet_path in dataframe["code_snippet"]]


    unsupervisedClassifier = KMeans(n_clusters=len(scopuri), random_state=0)
    unsupervisedClassifier.fit(embeddinguri)
    return unsupervisedClassifier
    

In [54]:
code_snip = "def s(a,b) return a+b"

In [48]:
clasificator = get_clasificator()

In [55]:

dataframe = citire_coduri()
set_scopuri = set([scop for scop in dataframe['scop']])
scopuri = [scop for scop in set_scopuri]
r = clasificator.predict([get_embedding_for_a_code_snippet(code_snip)[0]])[0]

print(scopuri[r])

Algoritmi pe siruri de numere


# PB 2

In [40]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

def generate_comments_with_model(code):
    # Tokenizează codul pentru a-l transforma în input-ul modelului
    tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
    inputs = tokenizer.encode("generate comment for code:\n" + code, return_tensors="pt", max_length=512, truncation=True)

    # Generează comentarii folosind modelul
    model = GPT2LMHeadModel.from_pretrained("gpt2")
    outputs = model.generate(inputs, max_length=150, num_return_sequences=1, temperature=0.8, do_sample=True, top_k=50)

    # Decodează rezultatul pentru a obține comentariul generat
    generated_comment = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return generated_comment

# Exemplu de cod Python fără comentarii
python_code_without_comments = """
def calculate_square(x):
    return x ** 2

result = calculate_square(5)
print(result)
"""

# Generare de comentarii pentru codul Python dat
generated_comment = generate_comments_with_model(python_code_without_comments)
print(generated_comment)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


generate comment for code:

def calculate_square(x):
    return x ** 2

result = calculate_square(5)
print(result)

So after some time the user found a function to calculate the square of the first input, and it was called in a block (or loop) of code.

The main method of this function is to generate a function in the form of a list. For example, the following code is generated from the following code:

def __init__(self, name=None):

return self

self.name = name

self.name.setDefault( " ")

self.name = name


